In [2]:
#%pip install git+https://github.com/ultralytics/ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics 8.3.50  Python-3.9.13 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 182.2/265.6 GB disk)


# Test Results

In [2]:
   import time
import torch
import os
from ultralytics import YOLO

# Define the models and their paths
models_info = [
    {
        "name": "Model_N_best.pt.pt",
        "weights_path": "Model/Model_N_best.pt"
    },
    {
        "name": "Model_S_best.pt.pt",
        "weights_path": "Model/Model_S_best.pt"
    },
    {
        "name": "Model_M_best.pt",
        "weights_path": "Model/Model_M_best.pt"
    }
]

# Path to the test images
test_images_dir = 'Dataset/test/images'

# Path to data.yaml
data_yaml_path = 'data.yaml'


# Get list of image files in the test directory
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# Initialize a list to store results
results = []

# Iterate over each model and evaluate
for model_info in models_info:
    model_name = model_info["name"]
    weights_path = model_info["weights_path"]
    
    # Load the YOLO model
    model = YOLO(weights_path)
    
    # -------------------- Model Evaluation --------------------
    # Evaluate the model on the test set
    metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

    # Extract metrics safely
    try:
        precision = metrics.box.p.mean()  # Mean Precision
        recall = metrics.box.r.mean()     # Mean Recall
        map50 = metrics.box.map50.mean()  # Mean mAP@50
        map = metrics.box.map.mean()      # Mean mAP@50-95
    except AttributeError:
        print(f"Error extracting metrics for {model_name}. Skipping...")
        continue

    # -------------------- Inference Time Calculation --------------------
    total_time = 0

    # Loop through each image file and perform inference
    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        # Suppress output from YOLO during inference
        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Perform inference with verbose=False
            inference_time = time.time() - start_time

        # Accumulate the inference time in milliseconds
        total_time += inference_time * 1000  # Convert to milliseconds

    # Calculate the average inference time in milliseconds
    average_inference_time = total_time / num_images

    # -------------------- Model Parameters and FLOPs --------------------
    # Extracting model parameters (in millions)
    params = sum(p.numel() for p in model.parameters()) / 1e6  # In millions

    # Assuming the model's FLOPs is roughly related to the number of parameters (this is an approximation)
    # Using a simple heuristic: FLOPs (billions) ~ Parameters (millions) * 2 (simplified estimate)
    flops = params * 2  # This is a rough approximation and may differ based on the architecture

    # Append the results for this model
    results.append({
        "Model": model_name,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "mAP@50": round(map50, 3),
        "mAP@50-95": round(map, 3),
        "Params(M)": round(params, 3),
        "FLOPs(B)": round(flops, 3),
        "Inference Time (ms)": round(average_inference_time, 2)
    })

# Print the results table
print(f"\n{'Model':<30}{'Precision':<10}{'Recall':<10}{'mAP@50':<10}{'mAP@50-95':<15}{'Params(M)':<10}{'FLOPs(B)':<10}{'Inference Time (ms)'}")
print("-" * 100)
for result in results:
    print(f"{result['Model']:<30}{result['Precision']:<10}{result['Recall']:<10}{result['mAP@50']:<10}{result['mAP@50-95']:<15}{result['Params(M)']:<10}{result['FLOPs(B)']:<10}{result['Inference Time (ms)']}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11n summary (fused): 238 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:11<00:00, 10.90it/s]


                   all       2033       4718       0.73      0.529      0.558      0.358
           boneanomaly         23         32      0.324      0.219      0.202     0.0674
            bonelesion          6          7          1       0.24      0.367      0.207
           foreignbody          1          1          1          0          0          0
              fracture       1346       1799      0.874      0.892      0.935      0.571
                 metal         66         73      0.859      0.986      0.984      0.837
    periostealreaction        223        351      0.608      0.632      0.639      0.292
          pronatorsign         42         42      0.513      0.357      0.509      0.299
            softtissue         50         53       0.43      0.455      0.391       0.21
                  text       2028       2360      0.959      0.981      0.991      0.739
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:12<00:00, 10.46it/s]


                   all       2033       4718      0.762      0.644       0.69      0.452
           boneanomaly         23         32      0.291      0.156      0.141     0.0594
            bonelesion          6          7          1      0.845      0.858      0.594
           foreignbody          1          1          1          0      0.497      0.254
              fracture       1346       1799      0.887      0.917      0.951      0.609
                 metal         66         73      0.919          1      0.986      0.851
    periostealreaction        223        351      0.683      0.681      0.718      0.351
          pronatorsign         42         42      0.564      0.833      0.666      0.389
            softtissue         50         53      0.555      0.377      0.398      0.216
                  text       2028       2360      0.964      0.982      0.992      0.746
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.24it/s]


                   all       2033       4718      0.796      0.706      0.816      0.554
           boneanomaly         23         32      0.635        0.5      0.466      0.244
            bonelesion          6          7       0.78      0.714      0.832      0.569
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1346       1799      0.925      0.959      0.978      0.671
                 metal         66         73       0.97          1      0.995      0.899
    periostealreaction        223        351      0.735      0.663      0.763      0.418
          pronatorsign         42         42       0.61      0.856      0.771      0.502
            softtissue         50         53       0.54      0.679      0.548      0.333
                  text       2028       2360      0.973      0.983      0.993      0.756
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

In [3]:
   import time
import torch
import os
from ultralytics import YOLO

# Define the models and their paths
models_info = [
    {
        "name": "Model_m_640_32_Adam_best.pt",
        "weights_path": "Model/Model_m_640_32_Adam_best.pt"
    },
    {
        "name": "Model_m_640_16_Adam_best.pt",
        "weights_path": "Model/Model_m_640_16_Adam_best.pt"
    },
    {
        "name": "Model_m_640_SGD_32_best.pt",
        "weights_path": "Model/Model_m_640_32_SGD_best.pt"
    },
    {
        "name": "Model_m_640_16_SGD_best.pt",
        "weights_path": "Model/Model_m_640_16_SGD_best.pt"
    }
]

# Path to data.yaml
data_yaml_path = 'data.yaml'

# Path to the test images
test_images_dir = 'Dataset/test/images'

# Get list of image files in the test directory
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# Initialize a list to store results
results = []

# Iterate over each model and evaluate
for model_info in models_info:
    model_name = model_info["name"]
    weights_path = model_info["weights_path"]
    
    # Load the YOLO model
    model = YOLO(weights_path)
    
    # -------------------- Model Evaluation --------------------
    # Evaluate the model on the test set
    metrics = model.val(data=data_yaml_path, split="test", imgsz=640)

    # Extract metrics safely
    try:
        precision = metrics.box.p.mean()  # Mean Precision
        recall = metrics.box.r.mean()     # Mean Recall
        map50 = metrics.box.map50.mean()  # Mean mAP@50
        map = metrics.box.map.mean()      # Mean mAP@50-95
    except AttributeError:
        print(f"Error extracting metrics for {model_name}. Skipping...")
        continue

    # -------------------- Inference Time Calculation --------------------
    total_time = 0

    # Loop through each image file and perform inference
    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        # Suppress output from YOLO during inference
        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Perform inference with verbose=False
            inference_time = time.time() - start_time

        # Accumulate the inference time in milliseconds
        total_time += inference_time * 1000  # Convert to milliseconds

    # Calculate the average inference time in milliseconds
    average_inference_time = total_time / num_images

    # -------------------- Model Parameters and FLOPs --------------------
    # Extracting model parameters (in millions)
    params = sum(p.numel() for p in model.parameters()) / 1e6  # In millions

    # Assuming the model's FLOPs is roughly related to the number of parameters (this is an approximation)
    # Using a simple heuristic: FLOPs (billions) ~ Parameters (millions) * 2 (simplified estimate)
    flops = params * 2  # This is a rough approximation and may differ based on the architecture

    # Append the results for this model
    results.append({
        "Model": model_name,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "mAP@50": round(map50, 3),
        "mAP@50-95": round(map, 3),
        "Params(M)": round(params, 3),
        "FLOPs(B)": round(flops, 3),
        "Inference Time (ms)": round(average_inference_time, 2)
    })

# Print the results table
print(f"\n{'Model':<30}{'Precision':<10}{'Recall':<10}{'mAP@50':<10}{'mAP@50-95':<15}{'Params(M)':<10}{'FLOPs(B)':<10}{'Inference Time (ms)'}")
print("-" * 100)
for result in results:
    print(f"{result['Model']:<30}{result['Precision']:<10}{result['Recall']:<10}{result['mAP@50']:<10}{result['mAP@50-95']:<15}{result['Params(M)']:<10}{result['FLOPs(B)']:<10}{result['Inference Time (ms)']}")


Ultralytics 8.3.50  Python-3.9.13 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.42it/s]


                   all       2033       4718      0.539      0.361      0.449      0.249
           boneanomaly         23         32          0          0    0.00383    0.00121
            bonelesion          6          7          1          0      0.423      0.248
           foreignbody          1          1          0          0          0          0
              fracture       1346       1799      0.871      0.856       0.91      0.541
                 metal         66         73       0.55       0.74      0.717      0.302
    periostealreaction        223        351      0.521      0.507      0.519      0.196
          pronatorsign         42         42      0.558      0.151      0.345      0.161
            softtissue         50         53      0.382     0.0189      0.135     0.0541
                  text       2028       2360      0.968      0.975      0.987      0.737
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.40it/s]


                   all       2033       4718      0.691      0.425      0.492      0.258
           boneanomaly         23         32      0.494      0.125      0.141     0.0337
            bonelesion          6          7          1          0      0.359      0.131
           foreignbody          1          1          1          0     0.0474     0.0237
              fracture       1346       1799      0.865      0.858      0.919      0.548
                 metal         66         73      0.454      0.822      0.724      0.317
    periostealreaction        223        351      0.514       0.53      0.547      0.218
          pronatorsign         42         42      0.518      0.262      0.429      0.206
            softtissue         50         53      0.416      0.245      0.269      0.106
                  text       2028       2360      0.961      0.981       0.99      0.738
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.18it/s]


                   all       2033       4718      0.747      0.686      0.797      0.557
           boneanomaly         23         32      0.642      0.438      0.445      0.249
            bonelesion          6          7      0.585      0.571      0.709      0.507
           foreignbody          1          1          1          0      0.995      0.796
              fracture       1346       1799      0.926      0.945      0.972      0.656
                 metal         66         73      0.923      0.988      0.993      0.849
    periostealreaction        223        351      0.791      0.677      0.798      0.434
          pronatorsign         42         42      0.459      0.952      0.786      0.501
            softtissue         50         53      0.425      0.623      0.481      0.267
                  text       2028       2360      0.973      0.981      0.991      0.754
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.11it/s]


                   all       2033       4718      0.796      0.706      0.816      0.554
           boneanomaly         23         32      0.635        0.5      0.466      0.244
            bonelesion          6          7       0.78      0.714      0.832      0.569
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1346       1799      0.925      0.959      0.978      0.671
                 metal         66         73       0.97          1      0.995      0.899
    periostealreaction        223        351      0.735      0.663      0.763      0.418
          pronatorsign         42         42       0.61      0.856      0.771      0.502
            softtissue         50         53       0.54      0.679      0.548      0.333
                  text       2028       2360      0.973      0.983      0.993      0.756
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

In [4]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'Model/Model_m_640_16_SGD_best.pt'
data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the test set
metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.31it/s]


                   all       2033       4718      0.796      0.706      0.816      0.554
           boneanomaly         23         32      0.635        0.5      0.466      0.244
            bonelesion          6          7       0.78      0.714      0.832      0.569
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1346       1799      0.925      0.959      0.978      0.671
                 metal         66         73       0.97          1      0.995      0.899
    periostealreaction        223        351      0.735      0.663      0.763      0.418
          pronatorsign         42         42       0.61      0.856      0.771      0.502
            softtissue         50         53       0.54      0.679      0.548      0.333
                  text       2028       2360      0.973      0.983      0.993      0.756
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

# Validation Results

In [5]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'Model/Model_m_640_16_SGD_best.pt'
data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the test set
metrics = model.val(data=data_yaml_path, split="val", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

Ultralytics 8.3.50  Python-3.9.13 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\val\labels.cache... 4066 images, 3 backgrounds, 0 corrupt: 100%|██████████| 4066/4066 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 255/255 [00:30<00:00,  8.31it/s]


                   all       4066       9426      0.755      0.743      0.788      0.548
           boneanomaly         33         46      0.713       0.63      0.712      0.452
            bonelesion         13         14      0.826      0.681      0.863      0.548
           foreignbody          2          2      0.519        0.5      0.497      0.398
              fracture       2709       3606      0.919      0.957      0.978      0.667
                 metal        140        163      0.891      0.975      0.986      0.865
    periostealreaction        441        693      0.747      0.733      0.772      0.429
          pronatorsign        102        102      0.686      0.728      0.824      0.517
            softtissue         77         82      0.527        0.5      0.467      0.306
                  text       4056       4718      0.969      0.985      0.992      0.753
Speed: 0.1ms preprocess, 3.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect